In [ ]:
#!pip install "basic-pitch[all]"

In [12]:
from basic_pitch.inference import predict
from basic_pitch import ICASSP_2022_MODEL_PATH

scikit-learn version 1.7.1 is not supported. Minimum required version: 0.17. Maximum required version: 1.5.1. Disabling scikit-learn conversion API.


In [13]:
# Reemplaza con la ruta de tu archivo de audio
audio_file_path = "test.wav"

# Realiza la predicción
model_output, midi_data, note_events = predict(audio_file_path, ICASSP_2022_MODEL_PATH)

# Los resultados incluyen:
# midi_data: El archivo MIDI transcrito.
# note_events: Una lista de eventos de notas (frecuencia, inicio, fin, etc.).
print(note_events)

Predicting MIDI for test.wav...
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
[(4.774268480725623, 4.960028117913832, 58, 0.8089292, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (4.774268480725623, 4.960028117913832, 46, 0.5512048, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (4.042839909297052, 4.774268480725623, 46, 0.61844885, [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), (3.9963999999999995, 4.774268480725623, 58, 0.7665925, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), (3.7745265306122446, 3.9486761904761902, 41, 0.

In [19]:
# Extraer solo las notas MIDI de la lista
midi_notes = [event[2] for event in note_events]
print(midi_notes)

[58, 46, 46, 58, 41, 41, 65, 55, 72, 60, 76, 79, 72, 56, 56, 80, 76, 45, 73, 79, 76, 72, 57, 79, 72, 60, 45, 79, 76, 72, 53, 76, 70, 77, 77, 82, 56]


In [20]:
from collections import Counter

# Usamos la lista 'midi_notes' que extrajimos en el paso anterior
midi_notes = [58, 46, 46, 58, 41, 41, 65, 55, 72, 60, 76, 79, 72, 56, 56, 80, 76, 45, 73, 79, 76, 72, 57, 79, 72, 60, 45, 79, 76, 72, 53, 76, 70, 77, 77, 82, 56]

# Contar la frecuencia de cada nota
note_counts = Counter(midi_notes)

# Obtener la nota más común
most_common_note = note_counts.most_common(1)

print(most_common_note)
# Resultado: [(76, 5)] -> La nota 76 se repitió 5 veces.

[(72, 5)]


In [23]:
# Mapeo de números de pitch class a nombres de notas
PITCH_CLASSES = [
    "C", "C#", "D", "D#", "E", "F", 
    "F#", "G", "G#", "A", "A#", "B"
]

In [31]:
# 5. Convertir el número MIDI a pitch class
pitch_class_number = most_common_note[0][0] % 12
pitch_class_name = PITCH_CLASSES[pitch_class_number]

print(f"La nota MIDI más común es: {most_common_note} (aparece {most_common_note[0][1]} veces)")
print(f"Su pitch class es: {pitch_class_name}")

La nota MIDI más común es: [(72, 5)] (aparece 5 veces)
Su pitch class es: C


---

In [48]:
k = 5 # number of most common notes

# Extract MIDI numbers from the note events
midi_notes = [int(event[2]) for event in note_events]

# Count the frequency of each MIDI note
note_counts = Counter(midi_notes)

# Get the top k most common notes
top_k_notes = note_counts.most_common(k)

results = []
print(f"Las {k} notas más comunes son:")
for note_midi, count in top_k_notes:
    # Convert MIDI number to pitch class
    pitch_class_number = note_midi % 12
    pitch_class_name = PITCH_CLASSES[pitch_class_number]
    
    results.append((pitch_class_name, note_midi, count))
    print(f"  - Nota: {pitch_class_name} (MIDI: {note_midi}) | Frecuencia: {count} veces")
    


Las 5 notas más comunes son:
  - Nota: C (MIDI: 72) | Frecuencia: 5 veces
  - Nota: E (MIDI: 76) | Frecuencia: 5 veces
  - Nota: G (MIDI: 79) | Frecuencia: 4 veces
  - Nota: G# (MIDI: 56) | Frecuencia: 3 veces
  - Nota: A# (MIDI: 58) | Frecuencia: 2 veces


In [70]:
def find_chord_from_notes(notes):
    """
    Identifies a major or minor chord from a list of notes.

    Args:
        notes (list): A list of three MIDI numbers.

    Returns:
        str: The name of the chord if a match is found, otherwise "No match".
    """
    # Sort notes for consistent interval calculation
    notes.sort()
    
    root_note = notes[0]
    second_note = notes[1]
    third_note = notes[2]

    # Calculate intervals in half-steps (semitones)
    interval_1 = second_note - root_note
    interval_2 = third_note - second_note

    # Check for a major chord (4, 3)
    if interval_1 == 4 and interval_2 == 3:
        return f"{get_pitch_class_name(root_note)} Major"

    # Check for a minor chord (3, 4)
    if interval_1 == 3 and interval_2 == 4:
        return f"{get_pitch_class_name(root_note)} Minor"

    return None

def get_pitch_class_name(midi_num):
    """Converts a MIDI number to its pitch class name."""
    PITCH_CLASSES = ["C", "C#", "D", "D#", "E", "F", 
                     "F#", "G", "G#", "A", "A#", "B"]
    return PITCH_CLASSES[midi_num % 12]

In [53]:
results

[('C', 72, 5), ('E', 76, 5), ('G', 79, 4), ('G#', 56, 3), ('A#', 58, 2)]

In [64]:
options = []
for note, midi, n in results:
    print(note, midi, sep=": ")
    options.append(midi)
print(options)

C: 72
E: 76
G: 79
G#: 56
A#: 58
[72, 76, 79, 56, 58]


In [ ]:
from itertools import combinations

In [65]:
chord_options = list(combinations(options, 3))
chord_options

[(72, 76, 79),
 (72, 76, 56),
 (72, 76, 58),
 (72, 79, 56),
 (72, 79, 58),
 (72, 56, 58),
 (76, 79, 56),
 (76, 79, 58),
 (76, 56, 58),
 (79, 56, 58)]

In [72]:
print("Possible tonalities:")
for chord in chord_options:
    found_chord = find_chord_from_notes(list(chord))
    if found_chord is not None:
        print(found_chord)

Possible tonalities:
C Major
